Yan Liang's homework

import the packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import *
from sklearn.metrics import *
from sklearn.model_selection import *

load the data

In [11]:
df = pd.read_csv ('day.csv')

print(df.describe())
print (df.head(5))

          instant      season          yr        mnth     holiday     weekday  \
count  730.000000  730.000000  730.000000  730.000000  730.000000  730.000000   
mean   365.500000    2.498630    0.500000    6.526027    0.028767    2.995890   
std    210.877136    1.110184    0.500343    3.450215    0.167266    2.000339   
min      1.000000    1.000000    0.000000    1.000000    0.000000    0.000000   
25%    183.250000    2.000000    0.000000    4.000000    0.000000    1.000000   
50%    365.500000    3.000000    0.500000    7.000000    0.000000    3.000000   
75%    547.750000    3.000000    1.000000   10.000000    0.000000    5.000000   
max    730.000000    4.000000    1.000000   12.000000    1.000000    6.000000   

       workingday  weathersit        temp       atemp         hum   windspeed  \
count  730.000000  730.000000  730.000000  730.000000  730.000000  730.000000   
mean     0.690411    1.394521   20.319259   23.726322   62.765175   12.763620   
std      0.462641    0.5448

conver the categorial varialbes to vectors

In [3]:
from sklearn.preprocessing import OneHotEncoder

def categorial_varable_to_vector_list(column_name):
    seasons = [[x] for x in df[column_name].unique()]
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(seasons)
    seasons_variable = [list(x) for x in list(enc.transform([[x] for x in df['season'].tolist()]).toarray())]
    return seasons_variable


categoarial_columns = ['season','yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']


x_all = []

for c in categoarial_columns:
    x = categorial_varable_to_vector_list(c)
    x = np.array(x).T
    x_all.append(x)

x_all = tuple(x_all)
x_categorail = np.concatenate(x_all).T

print(x_categorail.shape)

(730, 32)


convert the rest columns to vector

In [4]:
numemrical_columns = ['temp', 'atemp', 'hum', 'windspeed'] #, 'casual', 'registered'

x = [df[n].tolist() for n in numemrical_columns]

x_numerical = np.array(x).T

print(x_numerical.shape)

(730, 4)


combine the numberical and categorial varaibles

In [5]:
x = np.concatenate((x_categorail, x_numerical), axis=1)

print(x)

print(x.shape)

[[ 1.        0.        0.       ... 18.18125  80.5833   10.749882]
 [ 1.        0.        0.       ... 17.68695  69.6087   16.652113]
 [ 1.        0.        0.       ...  9.47025  43.7273   16.636703]
 ...
 [ 1.        0.        0.       ... 12.12     75.2917    8.333661]
 [ 1.        0.        0.       ... 11.585    48.3333   23.500518]
 [ 1.        0.        0.       ... 11.17435  57.75     10.374682]]
(730, 36)


prepare the output list

In [6]:
y = np.array(df['cnt'].tolist())

print(y.shape)

(730,)


split the training and test set

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state=42)

print(x_train.shape, y_train.shape)

print(x_test.shape, y_test.shape)

(657, 36) (657,)
(73, 36) (73,)


train the model

In [8]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(x_train, y_train)

print('Coefficients: \n', regr.coef_)

Coefficients: 
 [-4.50756382e+15  2.50853878e+15  2.11447898e+15  2.53740436e+15
  4.78409136e+13  2.08242100e+15  1.98143426e+15  1.66270434e+15
  1.79405761e+15  2.47962372e+15  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.98509009e+15
  0.00000000e+00  1.98509009e+15  1.66270434e+15  1.79405761e+15
  2.47962372e+15  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.98509009e+15  1.98509009e+15  1.66270434e+15  1.79405761e+15
 -7.97500000e+01  2.20250000e+02 -3.73750000e+01 -5.43437500e+01]


test the model

In [12]:
y_test_predict = regr.predict(x_test)

print('Mean squared error: %.2f'% mean_squared_error(y_test, y_test_predict))

print('Coefficient of determination: %.2f'% r2_score(y_test, y_test_predict))

Mean squared error: 2163455.29
Coefficient of determination: 0.31
